In [31]:
import requests
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

class GithubRepoApis:
    """ 
    Fetch the following data for a given username and (username-owned) repository.
    # endpoint for views
    endpoint = 'traffic/views'

    # endpoint for clones
    endpoint = 'traffic/clones'

    # endpoint for top referral references
    endpoint = 'traffic/popular/referrers'

    # endpoint for popular paths
    endpoint = 'traffic/popular/paths'

    Args:
        username (str): The GitHub username.
        repo (str): Name of username-owned repository
        token (str): Personal access token for authenticated requests - loaded from .env.

    Returns:
        nested dictionary: Containing data for each endpoint listed above.
    """
    
    def __init__(self, username: str = "neonwatty", repo: str = "meme-search"):
        # GitHub username and repository
        self.username = username
        self.repository = repo

        # Load the personal access token from .env
        self.access_token = os.getenv("GITHUB_TOKEN")
        if not self.access_token:
            raise ValueError("GitHub token not found. Please set GITHUB_TOKEN in the .env file.")

        # Form headers with authentication
        self.headers = {
            "Authorization": f"token {self.access_token}",
            "Accept": "application/vnd.github+json"
        }
        
        # List endpoints
        self.endpoints = ['traffic/views', 'traffic/clones', 'traffic/popular/referrers', 'traffic/popular/paths']

        # Container for endpoint data
        self.endpoint_data = {}
        
    def get_data(self, endpoint):
        # GitHub API URL for endpoint
        url = f"https://api.github.com/repos/{self.username}/{self.repository}/{endpoint}"

        # Make the GET request
        response = requests.get(url, headers=self.headers)

        # Check the response
        if response.status_code == 200:
            self.traffic_data = response.json()
            return self.traffic_data
        else:
            print(f"Failed to fetch data for endpoint {endpoint} - received: {response.status_code}")
            print(response.json())
            
    def get_all(self):
        # get all endpoint data
        for endpoint in self.endpoints:
            endpoint_name = endpoint.split("/")[-1]
            self.endpoint_data[endpoint_name] = self.get_data(endpoint)
            
            
def get_user_repositories(username: str = "neonwatty"):
    """
    Fetch the list of all repository names for a given GitHub username.

    Args:
        username (str): The GitHub username.
        token (str): Personal access token for authenticated requests - loaded from .env.

    Returns:
        list: A list of repository names.
    """
    # Load the personal access token from .env
    access_token = os.getenv("GITHUB_TOKEN")
    if not access_token:
        raise ValueError("GitHub token not found. Please set GITHUB_TOKEN in the .env file.")

    # Form headers with authentication
    headers = {
        "Authorization": f"token {access_token}",
        "Accept": "application/vnd.github+json"
    }
    
    # formulate url
    url = f"https://api.github.com/users/{username}/repos"

    # loop over pages and collect repo names    
    repos = []
    page = 1

    while True:
        try:
            # Fetch one page of repositories
            response = requests.get(url, headers=headers, params={"page": page, "per_page": 100})
            response.raise_for_status()  # Raise exception for HTTP errors
            data = response.json()

            if not data:
                break  # Exit loop if no more repositories

            # Append repository names to the list
            repos.extend(repo["name"] for repo in data)
            page += 1  # Go to the next page

        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            break

    return repos

In [ ]:
username = "neonwatty"
get_user_repositories(username)

In [28]:
data_fetcher = GithubRepoApis()
data_fetcher.get_all()

In [32]:
data_fetcher.endpoint_data

{'views': {'count': 195,
  'uniques': 53,
  'views': [{'timestamp': '2024-12-28T00:00:00Z', 'count': 30, 'uniques': 2},
   {'timestamp': '2024-12-29T00:00:00Z', 'count': 4, 'uniques': 2},
   {'timestamp': '2024-12-30T00:00:00Z', 'count': 3, 'uniques': 3},
   {'timestamp': '2024-12-31T00:00:00Z', 'count': 2, 'uniques': 2},
   {'timestamp': '2025-01-01T00:00:00Z', 'count': 8, 'uniques': 5},
   {'timestamp': '2025-01-02T00:00:00Z', 'count': 14, 'uniques': 10},
   {'timestamp': '2025-01-03T00:00:00Z', 'count': 5, 'uniques': 5},
   {'timestamp': '2025-01-04T00:00:00Z', 'count': 10, 'uniques': 4},
   {'timestamp': '2025-01-05T00:00:00Z', 'count': 12, 'uniques': 8},
   {'timestamp': '2025-01-06T00:00:00Z', 'count': 5, 'uniques': 3},
   {'timestamp': '2025-01-07T00:00:00Z', 'count': 38, 'uniques': 2},
   {'timestamp': '2025-01-08T00:00:00Z', 'count': 11, 'uniques': 6},
   {'timestamp': '2025-01-09T00:00:00Z', 'count': 6, 'uniques': 3},
   {'timestamp': '2025-01-10T00:00:00Z', 'count': 5, 'uniq

In [7]:
sum([v['uniques'] for v in data['views']])

64